In [7]:
from python_graphql_client import GraphqlClient
import pandas as pd
from datetime import datetime
import matplotlib

In [8]:
def client():
    return GraphqlClient(endpoint="https://api.thegraph.com/subgraphs/name/token-terminal-subgraphs/tornado-cash-v1-arbitrum")

def query(event, skip, timestamp):
    return """
    {
      """+event+"""(first: 1000, orderBy: blockTimestamp, skip: """+ str(skip)+""",
      where: {blockTimestamp_gt: """+ str(timestamp) +"""}) {
        id
        blockTimestamp
      }

    }
"""

In [9]:
def getPoolEvents(event, debug=False):
    timestamp=0
    poolEvents=0
    
    while(True):
        skip=0
        poolEventsTs=0
        while(skip <= 5000):
            querys = query(event, skip, timestamp)
            try:
                tgdata_raw = client().execute(query=querys)['data'][event]
                if debug : print(skip, timestamp, len(tgdata_raw))
            except:
                print(client().execute(query=querys))
                break
            tgdata = pd.DataFrame.from_dict(tgdata_raw)
            if type(poolEventsTs) is int : poolEventsTs = tgdata
            else : poolEventsTs = pd.concat([poolEventsTs,tgdata], ignore_index=True)

            if tgdata.shape[0] < 1000: break
            skip = skip + 1000
        if type(poolEvents) is int :  poolEvents = poolEventsTs 
        else :  poolEvents = pd.concat([poolEvents,poolEventsTs], ignore_index=True)

        if poolEventsTs.shape[0] < 6000: break
        timestamp = str(int(poolEventsTs['blockTimestamp'].iloc[-1]) -1)

    return poolEvents

poolEventsList =[]
for size in ['01','1','10','100']:
    deposits = getPoolEvents(f'eth{size}V1DepositV1Es').drop_duplicates()
    deposits['type'] = 'D'
    withdrawls = getPoolEvents(f'eth{size}V1WithdrawalV1Es').drop_duplicates()
    withdrawls['type'] = 'W'
    combined = pd.concat([deposits, withdrawls])
    combined['amount'] = size
    combined['currency'] = 'eth_arb'
    poolEventsList.append(combined)

poolEvents = pd.concat(poolEventsList)

data grouping

In [10]:
poolEvents['date']= pd.to_datetime(poolEvents['blockTimestamp'], unit='s').dt.date

In [11]:
dailyPoolEventsList = []
for date, df in poolEvents.groupby('date'):
    df['type'] = df['currency'] +'-'+ df['amount'] +'-'+ df['type']
    dayEvents = df.groupby('type')['id'].count().rename(date)
    dailyPoolEventsList.append(dayEvents)
dailyPoolEvents = pd.concat(dailyPoolEventsList, axis=1).sort_values(by='type').fillna(0).transpose()
cumPoolEvents = dailyPoolEvents.cumsum()
cumPoolEvents.to_json('daily-events-arbitrum.json')

In [12]:
import pandas as pd
eventDf = pd.read_json('daily-events-arbitrum.json')

In [13]:
eventDf

,eth_arb-01-D,eth_arb-01-W,eth_arb-1-D,eth_arb-1-W,eth_arb-10-D,eth_arb-10-W,eth_arb-100-D,eth_arb-100-W
2021-11-29,2,2,0,0,0,0,0,0
2021-11-30,8,4,2,0,0,0,0,0
2021-12-01,10,4,6,0,2,0,0,0
2021-12-02,12,6,16,0,8,0,0,0
2021-12-03,16,7,23,8,9,3,0,0
...,...,...,...,...,...,...,...,...
2022-04-18,651,529,1120,979,304,270,37,37
2022-04-19,664,540,1130,981,310,276,37,37
2022-04-20,672,542,1135,988,311,279,37,37
2022-04-21,697,568,1146,1005,316,285,37,37
